# Specs
dunno yet

In [20]:
# Imports
import sys
sys.path.append("../../nn-asr-resources/")
import data
from tensorflow import keras
import os
from time import strftime, gmtime

In [10]:
# Constants
_window_size = 20 # defines the past lookup for determining the following word
_path = "data/simple-examples/data"
_checkpoint_path = "training/conv2d/cp-{epoch:04d}_" + strftime("%H-%M_%d-%m-%Y", gmtime()) + ".ckpt"
_checkpoint_dir = os.path.dirname
_mask_value = 2.
features = 13

In [33]:
# Model creation
model = keras.models.Sequential()
model.add(keras.layers.Masking(_mask_value, input_shape=(None,_window_size, features, 1), name = "01-Masking_01"))
model.add(keras.layers.TimeDistributed(keras.layers.Conv2D(1,(1,2))))

TypeError: Layer conv2d_1 does not support masking, but was passed an input_mask: Tensor("time_distributed_1/Reshape_2:0", shape=(?, 20, 13), dtype=bool)

In [ ]:
model.summary()

In [22]:
BATCH_SIZE = 10
TIME_STEPS=86
INPUT_SIZE=28*28

model = Sequential()
model.add(InputLayer(batch_input_shape=(BATCH_SIZE, TIME_STEPS,     
INPUT_SIZE)))
print (model.output_shape)

model.add(TimeDistributed(Conv2D(64,(1,3),strides=(1,1), padding='same', 
data_format='channels_last')))  ##always the error here
print (model.output_shape)

model.add(TimeDistributed(MaxPooling2D(pool_size=(2,2),padding='same')))
print (model.output_shape)

model.add(TimeDistributed(Conv2D(64,(1,3),strides=(1,1), 
data_format='channels_last', padding='same')))
print (model.output_shape)

model.add(TimeDistributed(MaxPooling2D(pool_size=(2,2),padding='same')))
print (model.output_shape)

model.add(TimeDistributed(Flatten()))
print (model.output_shape)

model.add(TimeDistributed(Dense(4096, activation='relu')))
print (model.output_shape)

model.add(LSTM(100, stateful=True, return_sequences=True))
print (model.output_shape)

model.add(Dense(1, activation='sigmoid'))
print (model.output_shape)

ImportError: cannot import name 'Sequential'